In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress
from scipy.signal import butter,filtfilt, find_peaks

In [2]:
###### TO CHANGE
# Import data
# First: n = 270 & m = 780
# Second: n = 1145 & m = 1670
# Third: n = 2200 & m = 2800
# Fourth: n = 3350 & m = 4000
# Fifth: n = 4600 & m = 6500


n = 4630
m = 5780

# First: lim1 = 45 & lim2 = 70
# Second: lim1 = 57 & lim2 = 100
# Third: lim1 = 77 & lim2 = 120
# Fourth: lim1 = 90 & lim2 = 150
# Fifth: lim1 = 90 & lim2 = 175

lim1 = 77
lim2 = 120

disp = 0.5

In [3]:
df_exp_PB = pd.read_csv('PB20_TRI_16622.csv')                                                            # Read the file
df_exp_PB_1 = df_exp_PB.iloc[n:m]

stress1 = df_exp_PB_1['Stress [MPa]']             # Apply filter to stress
strain1 = df_exp_PB_1['Strain Ax [-]']            # Apply filter to strain

# print(stress1)

stress2 = stress1.to_numpy()
strain2 = strain1.to_numpy()


stress_range = stress1[(stress1 >= lim1) & (stress1 <= lim2)]                               # stress filtered for values more than 15 less 33
strain_range = strain1[(stress1 >= lim1) & (stress1 <= lim2)]                               # strain filtered for values more than 15 less 33

LVDT1_1 = df_exp_PB['LVDT1']
LVDT2_2 = df_exp_PB['LVDT2']
LVDT1 = LVDT1_1.to_numpy()
LVDT2 = LVDT2_2.to_numpy()
LVDT1_min = LVDT1.min()
LVDT2_min = LVDT2.min()
sample_length = 70
LVDT1_norm = np.array(())
LVDT2_norm = np.array(())
LVDT_avg = np.array(())
Strain_LVDT = np.array(())

for value in range(len(LVDT1)):
    LVDT1_norm = np.append(LVDT1_norm, LVDT1[value] - LVDT1_min)
    LVDT2_norm = np.append(LVDT2_norm, LVDT2[value] - LVDT2_min)

    LVDT_avg = np.append(LVDT_avg, (LVDT1_norm[value] + LVDT2_norm[value])/2)

    Strain_LVDT = np.append(Strain_LVDT, LVDT_avg[value]/sample_length)


# print(Strain_LVDT)

In [ ]:
# stress_range1 = stress_range.to_numpy()
# strain_range1 = strain_range.to_numpy()

stress_range1 = stress2
strain_range1 = strain2


CP = df_exp_PB_1['IPress']/10
CP1 = CP.to_numpy()
# print(CP1)

# Correction
# print(CP)
strain_rock = np.array(())
strain_machine = np.array(())
for value in range(len(stress_range1)):
    if (CP1[value] == 10):
        p = 3.671e+04
    elif (CP1[value] == 20):
        p = 4.305e+04
    elif (CP1[value] == 30):
        p = 4.8e+04
    elif (CP1[value] == 40):
        p = 5.107e+04
    elif (CP1[value] == 50):
        p = 5.316e+04

    strain_rock=np.append(strain_rock, strain_range1[value] - (stress_range1[value]/p))
    strain_machine=np.append(strain_machine, strain_range1[value] - strain_rock)

print(strain_rock)
# E_rock = linregress(strain_rock,stress_range1)
# print(E_rock)
# Linfit = E_rock[0]*(strain1) + E_rock[1]

corr_matrix = np.corrcoef(strain_rock, stress_range1)
corr = corr_matrix[0,1]
R_sq = corr**2
print(R_sq)

### Plot specific cycle
fig = go.Figure()
fig.add_trace(go.Scatter(name='Strain sample', x=strain_rock, y=stress_range1))
fig.add_trace(go.Scatter(name='Total Strain', x=strain1, y=stress1))
# fig.add_trace(go.Scatter(name='Strain Machine', x=strain_machine, y=stress_range1))
# fig.add_trace(go.Scatter(name='Strain Machine', x=strain1, y=Linfit))
fig.update_layout(template="simple_white", showlegend=True, autosize=False, width=700, height=700, 
    title="First cycle",
    xaxis_title="Strain Ax [-]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=20,
        color="Black"),
    legend=dict(title=None, orientation='h', y=1, yanchor="bottom", x=1, xanchor="right", font=dict(size = 16), bordercolor='black', borderwidth=1))

fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)

fig.show()

np.savetxt('sample.txt',strain_rock)

In [ ]:
strain_rock=np.array(())
E_al=70000  #MPa
stress_range1 = stress_range.to_numpy()
strain_range1 = strain_range.to_numpy()

for value in range(len(stress_range1)):
    if (stress_range1[value] >= lim1) & (stress_range1[value] <= lim2):
        p = 4.223e-06*(stress_range1[value])+0.0003365
    elif (stress_range1[value] >= lim1) & (stress_range1[value] <= lim2):
        p = 3.761e-06*(stress_range1[value])+0.0003662
    elif (stress_range1[value] >= lim1) & (stress_range1[value] <= lim2):
        p = 3.553e-06*(stress_range1[value])+0.000371
    elif (stress_range1[value] >= lim1) & (stress_range1[value] <= lim2):
        p = 3.371e-06*(stress_range1[value])+0.0003828
    elif (stress_range1[value] >= lim1) & (stress_range1[value] <= lim2):
        p = 3.294e-06*(stress_range1[value])+0.0003833


    strain_rock=np.append(strain_rock, strain_range1[value] - (1-1/E_al)*p)
# print(strain_rock)
# pd.DataFrame(strain_rock).to_csv("5cycle.csv")
E_rock = linregress(strain_rock,stress_range1)
print(E_rock)

In [ ]:
###ONLY FOR LINEAR REGRESSION
# Plot the young modulus slope
stress_range = stress1[(stress1 >= lim1) & (stress1 <= lim2)]                               # stress filtered for values more than 15 less 33
strain_range = strain1[(stress1 >= lim1) & (stress1 <= lim2)]                               # strain filtered for values more than 15 less 33


## Linear regression to fit the linear slope corresponding to the Young's
linear_regression_output = linregress(strain_range, stress_range)
print(linear_regression_output)
E = linear_regression_output[0]
b = linear_regression_output[1]
ElasticM = f'The elastic modulus is {round(E/1000,1)} GPa'

stress_offset = E*(strain1) + b - disp                                                                # y=mx+b


# # # print(len(strain1))
# # # # print(range(1,len(strain1)))
# # # print(len(stress_offset))
# # # print(len(stress1))

# ## Find yield strength
# for i in range(n, m):
#     if stress_offset[i] >= stress1[i]:
#         sx1 = strain1[i-1]
#         sy1 = stress1[i-1]
#         sy2 = stress1[i]
#         sx2 = strain1[i]

#         ox1 = strain1[i-1]
#         oy1 = stress_offset[i-1]
#         ox2 = stress1[i]
#         oy2 = stress_offset[i]
#         break

# # print(sx1, sy1, sx2, sy2, ox1, oy1, ox2, oy2)

# x1 = ox1
# y1 = oy1
# x2 = ox2
# y2 = oy2

# x3 = sx1
# y3 = sy1
# x4 = sx2
# y4 = sy2

# YS = ( (x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4-y3*x4) ) / ( ( (x1-x2)*(y3-y4))- ( (y1-y2)*(x3-x4)))
# print(YS)

# xi = (YS-b) / (E)
# yi = YS

# print('(xi,yi)',xi,yi)

In [ ]:
# # Maximum strength
# y = stress2.max()                                                                               # Find maximum stress
# max_index = stress2.argmax()
# # print(max_index)
# x = strain2[max_index]


# ## Fracture strength: point of strain where the material physically separates. 
# # At this point, the strain reaches its maximum value and the material actually fractures, even though the corresponding stress may be less than the ultimate strength at this point

# fract_strain =  strain2.max()
# max_index_fract = strain2.argmax()
# fract_stress = stress2[max_index_fract]
# # fract_stress = np.interp(fract_strain, strain1, stress1)
# FractS = f'The fracture strength is {round(fract_stress,1)} MPa'
# print(fract_strain, fract_stress)

In [ ]:
### Deviatoric stresses

# sigma1 = stress2
# sigma3 = np.zeros(len(stress2))
# # print(sigma3)
# for i in range(len(sigma1)):
#     if n >= 0 & m <=1300:
#         sigma3[i] = 10
#     elif n >= 2150 & m <=3400:
#         sigma3[i] = 20
#     elif n >= 4450 & m <=5800:
#         sigma3[i] = 30
#     elif n >= 7000 & m <=8500:
#         sigma3[i] = 40
#     else:
#         sigma3[i] = 50

# dev_stress = (sigma1 - sigma3)/2
# print(dev_stress)

In [5]:
### INTERACTIVE PLOTTING
## Plot all the data. This plots all the cycles together to visualize the data
fig = px.line(df_exp_PB, x = 'Strain Ax [-]', y = 'Stress [MPa]', hover_data=['Index'], width=700, height=700, title='PB20 - Full test', template="simple_white")
fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)
fig.update_layout( yaxis=dict(range=[0, 270]),# customize font and legend orientation & position
                  xaxis=dict(range=[0, 0.025]),
    font=dict(
        family="Times New Roman",
        size=20,
        color="Black"),
    legend=dict(title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"))

fig.show()

# fig.write_html("Graphs/Full_test.html")  

In [ ]:
### Plot loops
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=strain1, y=stress1))

# fig.update_layout(showlegend=False, autosize=False, width=700, height=700, 
#     title="PB20 LOOPs",
#     xaxis_title="Strain Ax [-]",
#     yaxis_title="Stress [MPa]",
#     font=dict(
#         family="New Times Roman",
#         size=15,
#         color="RebeccaPurple"))

# fig.show()

In [ ]:
### All info
# plt.plot(strain1,stress_offset)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=strain1, y=stress1))
# fig.add_trace(go.Scatter(x=strain1, y=stress_offset))

# ## Max strength
# fig.add_annotation(x=x, y=y,
#             text="Strain={:.4f}, Stress={:.4f}".format(x, y),
#             showarrow=True,
#             xanchor="right",
#             arrowhead=1)

# ## Elastic Moduli text
# fig.add_annotation(x=x, y=20,
#             text=ElasticM,
#             showarrow=False,
#             arrowhead=1)

# ## Fracture strength text
# fig.add_annotation(x=x, y=15,
#             text=FractS,
#             showarrow=False,
#             arrowhead=1)

# ## Yield point text
# fig.update_layout(showlegend=False, autosize=False, width=700, height=700, 
#     title="All info",
#     xaxis_title="Strain Ax [-]",
#     yaxis_title="Stress [MPa]",
#     font=dict(
#         family="New Times Roman",
#         size=15,
#         color="RebeccaPurple"))


# fig.add_annotation(x=xi, y=yi,
#             text="Strain={:.4f}, Stress={:.4f}".format(x1, y1),
#             xanchor="right",
#             showarrow=True,
#             arrowhead=1)

# fig.update_traces(textposition='top center')
# fig.show()

In [ ]:
## Plot all the data. This plots all the cycles together to visualize the data
time = df_exp_PB_1['Time']
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=time, y=stress1, name="Stress"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=time, y=strain1, name="Strain"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(template="simple_white", showlegend=True, autosize=False, width=800, height=700,
    yaxis=dict(range=[0, 270]),
    yaxis2=dict(range=[0, 0.025]),
    title_text="PB20", font=dict(
        family="Times New Roman",
        size=20,
        color="Black"),
    legend=dict(title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"))

# Set x-axis title
fig.update_xaxes(title_text="Time [sec]")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Stress [MPa]</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Strain [-]</b>", secondary_y=True)
fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)
fig.show()